## Setup

In [ ]:
# 1) Clone & cd
!git clone https://github.com/alonlahav123/DSC291-CLIP-Dissect.git
%cd DSC291-CLIP-Dissect

# # 2) Install PyTorch & torchvision
# !pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117

# 3) Install other deps
!pip install -r requirements.txt

# 3b) install these again (not sure why but this works)
!pip install tokenizer ftfy sentence-transformers huggingface-hub

# 4) Download the Broden dataset (images only) using
!bash dlbroden.sh

# 5) (Optional) Download Places-365 ResNet-18
# !bash dlzoo_example.sh

In [ ]:
# 1) upload imagenet_val 2012 after downloading from imagenet website
# 2) extract + remove the .tar.gz to save space

## Imagenet_val setup

In [ ]:
%%bash
cd data/imagenet_val

# Create the devkit folder if it doesn’t exist
mkdir -p ILSVRC2012_devkit_t12/data

# Download a known‐good meta.mat
wget -O ILSVRC2012_devkit_t12/data/meta.mat \
  https://raw.githubusercontent.com/calebrob6/imagenet_validation/master/data/meta.mat

# Download the ground-truth labels (50 000 integers 1–1000)
wget -O ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt \
  https://raw.githubusercontent.com/calebrob6/imagenet_validation/master/data/ILSVRC2012_validation_ground_truth.txt


# Need to organize the 50000 images into 1000 class folders (downloaded dataset is a flat folder with all the images)
python3 <<'EOF'
import os, numpy as np, scipy.io, shutil

# 1) Hardcode the full paths to the new files:
meta_mat_path = os.path.expanduser('~/private/DSC291-CLIP-Dissect/data/imagenet_val/ILSVRC2012_devkit_t12/data/meta.mat')
gt_path       = os.path.expanduser('~/private/DSC291-CLIP-Dissect/data/imagenet_val/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt')

# 2) Where your flat JPEGs live and where to write val_annotations.txt
flat_dir  = 'data/imagenet_val/images'
anns_file = 'data/imagenet_val/val_annotations.txt'

mat = scipy.io.loadmat(meta_mat_path, struct_as_record=False, squeeze_me=True)
wnids = [str(s.WNID) for s in mat['synsets']]

gt = np.loadtxt(gt_path, dtype=int)

imgs = sorted(f for f in os.listdir(flat_dir) if f.lower().endswith('.jpeg'))

# 3) Write val_annotations.txt linking each image to its WNID
with open(anns_file, 'w') as out:
    for img, lbl in zip(imgs, gt):
        out.write(f"{img}\t{wnids[lbl-1]}\n")

# 4) Move each JPEG into its synset subfolder
for line in open(anns_file):
    img, wnid = line.strip().split('\t')
    src = os.path.join(flat_dir, img)
    dst = os.path.join(flat_dir, wnid)
    os.makedirs(dst, exist_ok=True)
    shutil.move(src, os.path.join(dst, img))

print("✅ Done. Now have", len(os.listdir(flat_dir)), "folders under", flat_dir)
EOF

In [ ]:
# Check to see that all the images are classfiied. 
ls data/imagenet_val/images | wc -l   # should print “1000”